## Data Parsing

In [1]:
import os
import re
import time
import json
import httpx
import random
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from multiprocessing import Pool, cpu_count

In [2]:
lmap = lambda funcion, iterable: list(map(funcion, iterable))
lfilter = lambda funcion, iterable: list(filter(funcion, iterable))

In [3]:
data_dir = "../data/"
raw_dir = os.path.join(data_dir, "raw")
raw_csv_dir = os.path.join(raw_dir, "csv")
raw_json_dir = os.path.join(raw_dir, "json")
html_dir = os.path.join(raw_dir, "html")
completed_html_dir = os.path.join(html_dir, "completed")
upcoming_html_dir = os.path.join(html_dir, "upcoming")
fighterlist_html_dir = os.path.join(html_dir, "fighterlist")
fighters_html_dir = os.path.join(html_dir, "fighters")
completed_eventlist_html_dir = os.path.join(completed_html_dir, "eventlist")
completed_events_html_dir = os.path.join(completed_html_dir, "events")
completed_fights_html_dir = os.path.join(completed_html_dir, "fights")
upcoming_eventlist_html_dir = os.path.join(upcoming_html_dir, "eventlist")
upcoming_events_html_dir = os.path.join(upcoming_html_dir, "events")
upcoming_fights_html_dir = os.path.join(upcoming_html_dir, "fights")

In [4]:
dirs = [raw_csv_dir, raw_json_dir, fighters_html_dir, fighterlist_html_dir,
        completed_eventlist_html_dir, 
        completed_events_html_dir,
        upcoming_eventlist_html_dir, upcoming_events_html_dir,
       completed_fights_html_dir, upcoming_fights_html_dir]

for folderpath in dirs:
    os.makedirs(folderpath, exist_ok=True)

### Parse Fight page

In [5]:
def indiv_fight_data_extractor(fight_id_html):
    fight_id, fight_html = fight_id_html 
    fight_dict = {}

    soup = BeautifulSoup(fight_html)
    title_a_elem = soup.find("h2", class_="b-content__title").find("a")
    fight_dict["Event Name"] = title_a_elem.text.strip()
    fight_dict["Event Url"] = title_a_elem["href"]
    fight_dict["Fight ID"] = fight_id
    fighters = soup.find_all("div", class_="b-fight-details__person")
    for idx, fighter in enumerate(fighters, start=1):
        fight_dict[f"Fighter{idx} Status"] = fighter.find("i", class_="b-fight-details__person-status").text.strip()

        a_elem = fighter.find("a", class_="b-link b-fight-details__person-link")
        fight_dict[f"Fighter{idx} Name"] = a_elem.text.strip()
        fight_dict[f"Fighter{idx} Url"] = a_elem["href"]

    fight_dict["Bout"] = soup.find("i", class_="b-fight-details__fight-title").text.strip()

    fight_details_div = soup.find("div", class_="b-fight-details__content")

    method_elem = fight_details_div.find("p").find("i", class_="b-fight-details__text-item_first")

    def get_details(i_elem):
        i_text = i_elem.text.replace("\n", "").strip()
        m = re.search(r"(.*):\s+(.*)", i_text)
        if m:
            return (m.group(1), m.group(2))

    label_elems = fight_details_div.find_all("i", class_="b-fight-details__label")
    detail_elems = lmap(lambda e: e.parent, label_elems)
    detail_tups = lfilter(lambda t: t != None, map(get_details, detail_elems))

    for label, text in detail_tups:
        fight_dict[label] = text

    fight_dict

    details_text = fight_details_div.find_all("p")[1].text.replace("\n", "").strip()

    m = re.search(r"(.*):\s+(.*)", details_text)
    if m:
        fight_dict[m.group(1)] = m.group(2)

    fight_dict

    tables = soup.find_all("table")

    len(tables)

    def fight_tables_to_dicts(page_html: str):

        def extract_table(tables):
            data_dict = {key:[] for key in map(lambda x: x.text.strip(), tables[0].find("thead").find_all("th"))}
            data_dict["Round"] = []

            for i, table in enumerate(tables[:2]):
                rows = table.find("tbody").find_all("tr")

                for j, row in enumerate(rows, start=1):
                    for col, elem in zip(data_dict.keys(), row.find_all("td")):
                        if col == "Fighter":
                            for a_elem in elem.find_all("a"):
                                data_dict[col].append(a_elem.text.strip())
                        else:
                            for p_elem in elem.find_all("p"):
                                val = p_elem.text.strip()
                                val = val if val != "---" else None
                                data_dict[col].append(val)

                    if i == 0:
                        data_dict["Round"].extend(["Overall", "Overall"])
                    else:
                        data_dict["Round"].extend([f"Round {j}", f"Round {j}"])


            return data_dict


        soup = BeautifulSoup(page_html, features="lxml")
        tables = soup.find_all("table")
        
        return extract_table(tables[:2]), extract_table(tables[2:])
         

    try:
        fight_dict["Totals"], fight_dict["Significant Strikes"] = fight_tables_to_dicts(fight_html)
    except IndexError:
        print("Table IndexError")
        pass 
    
    return fight_dict

In [6]:
def all_fight_data_extractor(fights_html_dir):
    fights_html_dict = {}

    fight_files = lfilter(lambda x: x.endswith(".html"), os.listdir(fights_html_dir))
    for filename in tqdm(fight_files):
        filepath = os.path.join(fights_html_dir, filename)
        with open(filepath, "r") as f:
            html_str = f.read()
            fight_id = filename.replace(".html","")
            fights_html_dict[fight_id] = html_str

    # fights_dict_list = []
    
    # for fight_id, fight_html in tqdm(fights_html_dict.items()):
    #     fights_dict_list.append(indiv_fight_data_extractor(fight_id, fight_html))
    
    with Pool(cpu_count()) as p:
        fights_dict_list = p.map(indiv_fight_data_extractor, fights_html_dict.items())
        
    return fights_dict_list

In [7]:
completed_fights_dict_list = all_fight_data_extractor(completed_fights_html_dir)

100%|██████████| 6875/6875 [00:00<00:00, 15582.35it/s]


Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError


In [8]:
len(completed_fights_dict_list)

6875

In [13]:
filepath = os.path.join(raw_json_dir, "completed_fights_data.json")
with open(filepath, "w") as f:
    json.dump(completed_fights_dict_list, f, indent=4)

### Parse Fighter pages

In [14]:
def extract_fighter_data(fighter_page_filepath):

    with open(fighter_page_filepath, "r") as f:
        html_str = f.read()

    soup = BeautifulSoup(html_str)

    fighter_dict = dict()

    fighter_dict["Name"] = soup.find("span", class_="b-content__title-highlight").text.strip() 

    fighter_dict["Fighter ID"] = os.path.split(fighter_page_filepath)[1].replace(".html","")
    
    fighter_dict["Record"] = soup.find("span", class_="b-content__title-record").text.replace("Record:","").strip() 

    nn = soup.find("p", class_="b-content__Nickname").text.strip()
    fighter_dict["Nickname"] = nn if nn else None

    fighter_dict

    uls = soup.find_all("ul", class_="b-list__box-list")

    info_list = [li.text.replace("\n","").strip() for ul in uls for li in ul.find_all("li")]

    info_list = lfilter(lambda s: s, info_list)

    for info in info_list:
        m = re.search(r"(.*):\s+(.*)", info)
        if m:
            key, val = m.group(1).strip(), m.group(2).strip()
            if key == "Height":
                val = val.replace(" ","")
            if val == "--":
                val = None
            fighter_dict[key] = val

    return fighter_dict

In [15]:
def create_fighters_df(fighters_html_dir):
    fighter_files = os.listdir(fighters_html_dir)
    fighter_files = lfilter(lambda s: s.endswith(".html"), fighter_files)
    fighter_filepaths = lmap(lambda s: os.path.join(fighters_html_dir, s), fighter_files)
    
    with Pool(cpu_count()) as p:
        fighters_lod = p.map(extract_fighter_data, fighter_filepaths)
        
    return pd.DataFrame(fighters_lod)

In [16]:
fighters_df = create_fighters_df(fighters_html_dir)

In [17]:
fighters_df.head()

,Name,Fighter ID,Record,Nickname,Height,Weight,Reach,STANCE,DOB,SLpM,Str. Acc.,SApM,Str. Def,TD Avg.,TD Acc.,TD Def.,Sub. Avg.
0,Devin Clark,cc7040fe76f0ef91,13-7-0,Brown Bear,"6'0""",205 lbs.,"75""",Orthodox,"Apr 12, 1990",2.94,55%,2.86,44%,2.30,35%,55%,0.1
1,Chuck Liddell,a390eb8a9b2df298,21-8-0,The Iceman,"6'2""",205 lbs.,"76""",Orthodox,"Dec 17, 1969",2.97,35%,2.71,56%,0.44,75%,81%,0.0
2,Mike King,564c2f5f9eb45d67,5-1-0,None,"6'3""",185 lbs.,None,NaN,"Aug 04, 1983",4.44,58%,1.29,72%,1.75,25%,40%,3.5
3,Kenji Arai,269d103c96a4c3a5,15-15-5,None,"5'10""",145 lbs.,None,Southpaw,"Oct 05, 1979",8.79,27%,10.29,39%,0.00,0%,0%,0.0
4,Jalin Turner,4d8fa64202cee4c1,13-5-0,The Tarantula,"6'3""",155 lbs.,"77""",Southpaw,"May 18, 1995",6.48,47%,4.31,44%,1.14,80%,77%,2.0


In [18]:
filepath = os.path.join(raw_csv_dir, "fighters_data.csv")
fighters_df.to_csv(filepath, index=False)